# Pytorch Training UI

### Import Libraries

In [1]:
# detect whether this is a google environment

COLAB_ENVIRONMENT = False

try:
    from google.colab import drive
    drive.mount('/content/drive')
    COLAB_ENVIRONMENT = True
except:
    pass

In [2]:
import sys
import os
import pickle

if COLAB_ENVIRONMENT:
    py_file_location = "./drive/MyDrive/LAB/COMP90051-A1__Groupwork__Py/PrivatePackages/pytorch" # my private packages are stored here
    home_directory = './drive/MyDrive/LAB/COMP90051-A1__Groupwork__Py/' # my home directory is stored in ./LAB of google drive
    !pip install einops
else:
    py_file_location = './PrivatePackages/pytorch'
    home_directory = './'

sys.path.append(os.path.abspath(py_file_location))

from environment import *
from utils import *

from sklearn.model_selection import train_test_split

In [3]:
from model.model_class import LSTM, BERT, LSTM_DANN, BERT_DANN, LSTM_DCE_DANN, BERT_DCE_DANN, LSTM_Hinge, BERT_Hinge

### Set Seed and Load Data

In [4]:
SEED = 2608

In [5]:
data1 = []
with open(home_directory + '/data/raw/comp90051-2024s1-project-1/domain1_train_data.json', 'r') as f:
    for line in f:
        data1.append(json.loads(line))

data2 = []
with open(home_directory + './data/raw/comp90051-2024s1-project-1/domain2_train_data.json', 'r') as f:
    for line in f:
        data2.append(json.loads(line))

data_test = []
with open(home_directory + '/data/raw/comp90051-2024s1-project-1/test_data.json', 'r') as f:
    for line in f:
        data_test.append(json.loads(line))

# create domain labels for data
for i in range(len(data1)):
    data1[i]['domain'] = 0
for i in range(len(data2)):
    data2[i]['domain'] = 1

In [6]:
# Train Val Test Split

# get labels for stratification
label1 = [instance['label'] for instance in data1]
label2 = [instance['label'] for instance in data2]

train_ix_1, val_test_ix_1 = train_test_split(range(len(data1)), test_size=0.3, random_state=SEED, stratify = label1)
train_ix_2, val_test_ix_2 = train_test_split(range(len(data2)), test_size=0.3, random_state=SEED, stratify = label2)
val_ix_1, test_ix_1 = train_test_split(val_test_ix_1, test_size=0.5, random_state=SEED, stratify = [data1[i]['label'] for i in val_test_ix_1])
val_ix_2, test_ix_2 = train_test_split(val_test_ix_2, test_size=0.5, random_state=SEED, stratify = [data2[i]['label'] for i in val_test_ix_2])

# split data according to the index from train_test_split
train_data_1 = [data1[i] for i in train_ix_1]
val_data_1 = [data1[i] for i in val_ix_1]
test_data_1 = [data1[i] for i in test_ix_1]
train_data_2 = [data2[i] for i in train_ix_2]
val_data_2 = [data2[i] for i in val_ix_2]
test_data_2 = [data2[i] for i in test_ix_2]

# combine the data
train_data = train_data_1 + train_data_2
val_data = val_data_1 + val_data_2
test_data = test_data_1 + test_data_2

---
### Models

#### 1. Pure Classification

In [7]:
# BERT - CELoss

MAX_SENTENCE_LENGTH = 256
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 1024 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 8
    embedding_aggregation = 'cls' # TODO
    n_mlp_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 10
    loss = nn.CrossEntropyLoss()
    # loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    validation_loss = nn.CrossEntropyLoss()
    # validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Classifier'

model = BERT(BERT_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
# pretrain_best_epoch = model.fit_pretrain(pretrain_x, pretrain_y, pretrain_dom, pretrain_mask, preval_x, preval_y, preval_dom, preval_mask)
# print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
# model.load()
# model.eval_pretrain(preval_x, preval_y, preval_dom, preval_mask, pretrain_best_epoch, evaluation_mode = True)

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

 11%|█▏        | 453/4000 [00:00<00:00, 64558.45it/s]


KeyboardInterrupt: 

In [ ]:
# BERT - WCELoss

MAX_SENTENCE_LENGTH = 256
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 1024 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 8
    embedding_aggregation = 'cls' # TODO
    n_mlp_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 10
    # loss = nn.CrossEntropyLoss()
    loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.CrossEntropyLoss()
    validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Classifier'

model = BERT(BERT_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
# pretrain_best_epoch = model.fit_pretrain(pretrain_x, pretrain_y, pretrain_dom, pretrain_mask, preval_x, preval_y, preval_dom, preval_mask)
# print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
# model.load()
# model.eval_pretrain(preval_x, preval_y, preval_dom, preval_mask, pretrain_best_epoch, evaluation_mode = True)

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

In [ ]:
# model.load()
# model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
# model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

# EXPERIMENT_NAME = '8bert_1mlpc_1mlpd_256d1024_256d40_8h_0.1_embed_4batch_bwce'

# future_pred_y = model.predict(future_x)

# future_pred_y = [1 if x[1] > x[0] else 0 for x in future_pred_y]

# predictions = pd.DataFrame({'id': range(len(future_pred_y)), 'class': future_pred_y})
# predictions.to_csv(home_directory + f'predictions/{EXPERIMENT_NAME}_classification.csv', index=False)

Epoch 20 Val   | Loss:  0.2023 | Accuracy:  0.8459| F1:  0.7007 | Balanced Accuracy:  0.8337 | Dom Avg Accuracy:  0.7931 |
                Domain 1 Accuracy:  0.7587| Domain 1 F1:  0.7774 | Domain 1 Balanced Accuracy:  0.7587 | 
                Domain 2 Accuracy:  0.8795| Domain 2 F1:  0.5927 | Domain 2 Balanced Accuracy:  0.8275
Epoch 20 Val   | Loss:  0.2083 | Accuracy:  0.8385| F1:  0.6845 | Balanced Accuracy:  0.8206 | Dom Avg Accuracy:  0.7759 |
                Domain 1 Accuracy:  0.7320| Domain 1 F1:  0.7528 | Domain 1 Balanced Accuracy:  0.7320 | 
                Domain 2 Accuracy:  0.8795| Domain 2 F1:  0.5870 | Domain 2 Balanced Accuracy:  0.8198


---
#### 2. DANN

In [ ]:
# BERT - DANN CELoss

MAX_SENTENCE_LENGTH = 256
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 1024 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 8
    embedding_aggregation = 'cls' # TODO
    n_mlp_clf_layers = 1
    n_mlp_dom_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 10
    # loss = nn.CrossEntropyLoss()
    loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.CrossEntropyLoss()
    validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    domain_loss = nn.CrossEntropyLoss()
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    alpha = 0.1
    gradient_reversal_every_n_epoch = 1
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_DANN'

model = BERT_DANN(BERT_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

In [ ]:
# BERT - DANN WCELoss

MAX_SENTENCE_LENGTH = 256
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 1024 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 8
    embedding_aggregation = 'cls' # TODO
    n_mlp_clf_layers = 1
    n_mlp_dom_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 10
    # loss = nn.CrossEntropyLoss()
    loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.CrossEntropyLoss()
    validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    domain_loss = nn.CrossEntropyLoss()
    alpha = 0.1
    gradient_reversal_every_n_epoch = 1
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_DANN'

model = BERT_DANN(BERT_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

In [ ]:
# EXPERIMENT_NAME = '8bert_1mlpc_1mlpd_256d1024_256d40_8h_0.1_embed_4batch_bwce_low_freq'

# future_pred_y, future_pred_dom = model.predict(future_x)

# future_pred_y = [1 if x[1] > x[0] else 0 for x in future_pred_y]

# predictions = pd.DataFrame({'id': range(len(future_pred_y)), 'class': future_pred_y})
# predictions.to_csv(home_directory + f'predictions/{EXPERIMENT_NAME}_classification.csv', index=False)

100%|██████████| 4000/4000 [00:00<00:00, 39010.34it/s]


class prior: 0.2222222222222222 0.7777777777777778
domain prior: 0.7222222222222222 0.2777777777777778
dom1 class prior: 0.5 0.5
dom2 class prior: 0.11538461538461539 0.8846153846153846


100%|██████████| 2700/2700 [00:00<00:00, 6566.89it/s]


---
Epoch 21 Val   | Classification Loss:  0.1625 | Accuracy:  0.8574| F1:  0.7150 | Balanced Accuracy:  0.8387 | Dom Avg Accuracy:  0.8010 |
                            Domain Loss:  0.5397 | Domain Accuracy:  0.7581 |  
                            Domain 1 Accuracy:  0.7920| Domain 1 F1:  0.8050 | Domain 1 Balanced Accuracy:  0.7920 |  
                            Domain 2 Accuracy:  0.8826| Domain 2 F1:  0.5844 | Domain 2 Balanced Accuracy:  0.8100
Epoch 21 Val   | Classification Loss:  0.1658 | Accuracy:  0.8589| F1:  0.7167 | Balanced Accuracy:  0.8390 | Dom Avg Accuracy:  0.8013 |
                            Domain Loss:  0.5434 | Domain Accuracy:  0.7607 |  
                            Domain 1 Accuracy:  0.7667| Domain 1 F1:  0.7804 | Domain 1 Balanced Accuracy:  0.7667 |  
                            Domain 2 Accuracy:  0.8944| Domain 2 F1:  0.6241 | Domain 2 Balanced Accuracy:  0.8359


---
#### 3. DCE_DANN

In [ ]:
# BERT DBCE - celoss expr - just bal domain

MAX_SENTENCE_LENGTH = 256
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_DCE_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 1024 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 8
    embedding_aggregation = 'cls' # TODO
    n_mlp_clf_layers = 1
    n_mlp_dom_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 10
    # loss = nn.CrossEntropyLoss()
    loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.CrossEntropyLoss()
    validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    domain_1_loss = nn.CrossEntropyLoss()
    # domain_1_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([dom1_pos_prior, dom1_neg_prior]))
    domain_1_validation_loss = nn.CrossEntropyLoss()
    # domain_1_validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([dom1_pos_prior, dom1_neg_prior]))
    domain_2_loss = nn.CrossEntropyLoss()
    # domain_2_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([dom2_pos_prior, dom2_neg_prior]))
    domain_2_validation_loss = nn.CrossEntropyLoss()
    # domain_2_validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([dom2_pos_prior, dom2_neg_prior]))
    domain_prior = [pos_dom_prior, neg_dom_prior]
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    domain_loss = nn.CrossEntropyLoss()
    alpha = 0.1
    gradient_reversal_every_n_epoch = 1
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Classifier'

model = BERT_DCE_DANN(BERT_DCE_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

In [ ]:
# BERT DBCE - celoss expr - just bal dom label

MAX_SENTENCE_LENGTH = 256
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_DCE_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 1024 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 8
    embedding_aggregation = 'cls' # TODO
    n_mlp_clf_layers = 1
    n_mlp_dom_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 10
    # loss = nn.CrossEntropyLoss()
    loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.CrossEntropyLoss()
    validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # domain_1_loss = nn.CrossEntropyLoss()
    domain_1_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([dom1_pos_prior, dom1_neg_prior]))
    # domain_1_validation_loss = nn.CrossEntropyLoss()
    domain_1_validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([dom1_pos_prior, dom1_neg_prior]))
    # domain_2_loss = nn.CrossEntropyLoss()
    domain_2_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([dom2_pos_prior, dom2_neg_prior]))
    # domain_2_validation_loss = nn.CrossEntropyLoss()
    domain_2_validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([dom2_pos_prior, dom2_neg_prior]))
    domain_prior = [0.5, 0.5]
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    domain_loss = nn.CrossEntropyLoss()
    alpha = 0.1
    gradient_reversal_every_n_epoch = 1
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Classifier'

model = BERT_DCE_DANN(BERT_DCE_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

In [ ]:
# BERT DBCE - celoss expr - bal both domain and label

MAX_SENTENCE_LENGTH = 256
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_DCE_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 1024 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 8
    embedding_aggregation = 'cls' # TODO
    n_mlp_clf_layers = 1
    n_mlp_dom_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 10
    # loss = nn.CrossEntropyLoss()
    loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.CrossEntropyLoss()
    validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # domain_1_loss = nn.CrossEntropyLoss()
    domain_1_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([dom1_pos_prior, dom1_neg_prior]))
    # domain_1_validation_loss = nn.CrossEntropyLoss()
    domain_1_validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([dom1_pos_prior, dom1_neg_prior]))
    # domain_2_loss = nn.CrossEntropyLoss()
    domain_2_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([dom2_pos_prior, dom2_neg_prior]))
    # domain_2_validation_loss = nn.CrossEntropyLoss()
    domain_2_validation_loss = nn.CrossEntropyLoss(weight=torch.FloatTensor([dom2_pos_prior, dom2_neg_prior]))
    domain_prior = [pos_dom_prior, neg_dom_prior]
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    domain_loss = nn.CrossEntropyLoss()
    alpha = 0.1
    gradient_reversal_every_n_epoch = 1
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Classifier'

model = BERT_DCE_DANN(BERT_DCE_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

100%|██████████| 4000/4000 [00:00<00:00, 53698.43it/s]


class prior: 0.2222222222222222 0.7777777777777778
domain prior: 0.7222222222222222 0.2777777777777778
dom1 class prior: 0.5 0.5
dom2 class prior: 0.11538461538461539 0.8846153846153846


100%|██████████| 2700/2700 [00:00<00:00, 6401.98it/s]


---


100%|█████████▉| 1575/1576 [01:04<00:00, 24.47it/s]


...

Epoch 24 Train | Loss:  0.0406 | Accuracy:  0.8390| F1:  0.7132 | Balanced Accuracy:  0.8611 | Dom Avg Accuracy:  0.8719 |
                    Domain 1 Accuracy:  0.8877| Domain 1 F1:  0.8891 | Domain 1 Balanced Accuracy:  0.8877 | 
                    Domain 2 Accuracy:  0.8202| Domain 2 F1:  0.5368 | Domain 2 Balanced Accuracy:  0.8561
Epoch 24 Val   | Loss:  0.1700 | Accuracy:  0.7911| F1:  0.6250 | Balanced Accuracy:  0.7883 | Dom Avg Accuracy:  0.7846 |
                Domain 1 Accuracy:  0.7720| Domain 1 F1:  0.7729 | Domain 1 Balanced Accuracy:  0.7720 | 
                Domain 2 Accuracy:  0.7985| Domain 2 F1:  0.4767 | Domain 2 Balanced Accuracy:  0.7972


100%|█████████▉| 1575/1576 [01:04<00:00, 24.50it/s]


Epoch 25 Train | Loss:  0.0379 | Accuracy:  0.8463| F1:  0.7254 | Balanced Accuracy:  0.8704 | Dom Avg Accuracy:  0.8811 |
                    Domain 1 Accuracy:  0.9034| Domain 1 F1:  0.9050 | Domain 1 Balanced Accuracy:  0.9034 | 
                    Domain 2 Accuracy:  0.8243| Domain 2 F1:  0.5428 | Domain 2 Balanced Accuracy:  0.8589
Epoch 25 Val   | Loss:  0.1830 | Accuracy:  0.7489| F1:  0.5891 | Balanced Accuracy:  0.7707 | Dom Avg Accuracy:  0.7707 |
                Domain 1 Accuracy:  0.7467| Domain 1 F1:  0.7558 | Domain 1 Balanced Accuracy:  0.7467 | 
                Domain 2 Accuracy:  0.7497| Domain 2 F1:  0.4404 | Domain 2 Balanced Accuracy:  0.7948


100%|█████████▉| 1575/1576 [01:04<00:00, 24.55it/s]


Epoch 26 Train | Loss:  0.0373 | Accuracy:  0.8544| F1:  0.7369 | Balanced Accuracy:  0.8770 | Dom Avg Accuracy:  0.8878 |
                    Domain 1 Accuracy:  0.9086| Domain 1 F1:  0.9098 | Domain 1 Balanced Accuracy:  0.9086 | 
                    Domain 2 Accuracy:  0.8335| Domain 2 F1:  0.5579 | Domain 2 Balanced Accuracy:  0.8670
Epoch 26 Val   | Loss:  0.1916 | Accuracy:  0.7333| F1:  0.5804 | Balanced Accuracy:  0.7679 | Dom Avg Accuracy:  0.7759 |
                Domain 1 Accuracy:  0.7507| Domain 1 F1:  0.7599 | Domain 1 Balanced Accuracy:  0.7507 | 
                Domain 2 Accuracy:  0.7267| Domain 2 F1:  0.4312 | Domain 2 Balanced Accuracy:  0.8011

Epoch 24 Val   | Loss:  0.1700 | Accuracy:  0.7911| F1:  0.6250 | Balanced Accuracy:  0.7883 | Dom Avg Accuracy:  0.7846 |
                Domain 1 Accuracy:  0.7720| Domain 1 F1:  0.7729 | Domain 1 Balanced Accuracy:  0.7720 | 
                Domain 2 Accuracy:  0.7985| Domain 2 F1:  0.4767 | Domain 2 Balanced Accuracy:  0.

---
# Hinge Loss

In [7]:
# BERT - hinge work in progress

MAX_SENTENCE_LENGTH = 256
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)

# train_x = train_x[:8]
# train_y = train_y[:8]
# train_dom = train_dom[:8]
# val_dom = val_dom[:8]
# val_x = val_x[:8]
# val_y = val_y[:8]

pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_Hinge_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 1024 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 6
    embedding_aggregation = 'cls' # TODO
    n_mlp_layers = 1
    # n_mlp_dom_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-3
    patience = 10
    # loss = nn.BCELoss()
    # loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    loss = HingeLoss()
    # validation_loss = nn.BCELoss()
    # validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    validation_loss = HingeLoss()
    domain_loss = nn.BCELoss()
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    alpha = 0
    gradient_reversal_every_n_epoch = 1
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 1
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_Hinge'

model = BERT_Hinge(BERT_Hinge_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

100%|██████████| 4000/4000 [00:00<00:00, 29364.68it/s]


class prior: 0.75 0.25
domain prior: 0.0 1.0
dom1 class prior: 0.5 0.5
dom2 class prior: 0.11538461538461539 0.8846153846153846


100%|██████████| 2700/2700 [00:00<00:00, 4334.59it/s]


---


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
 50%|█████     | 1/2 [00:01<00:01,  1.79s/it]
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitali

...

Epoch 3 Train | Loss:  0.5789 | Accuracy:  0.7500| F1:  0.8571 | Balanced Accuracy:  0.5000 | Dom Avg Accuracy:     nan |
                    Domain 1 Accuracy:  0.7500| Domain 1 F1:  0.8571 | Domain 1 Balanced Accuracy:  0.5000 | 
                    Domain 2 Accuracy:     nan| Domain 2 F1:  0.0000 | Domain 2 Balanced Accuracy:     nan


/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Applications/anaconda3/envs/COMP90051/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarni

Epoch 3 Val   | Loss:  1.2411 | Accuracy:  0.3750| F1:  0.5455 | Balanced Accuracy:  0.5000 | Dom Avg Accuracy:     nan |
                Domain 1 Accuracy:  0.3750| Domain 1 F1:  0.5455 | Domain 1 Balanced Accuracy:  0.5000 | 
                Domain 2 Accuracy:     nan| Domain 2 F1:  0.0000 | Domain 2 Balanced Accuracy:     nan


  0%|          | 0/2 [00:02<?, ?it/s]


KeyboardInterrupt: 

---

# Historic Best

In [ ]:
# BERT - SoftmaxBCELoss+OptBalAccu

MAX_SENTENCE_LENGTH = 256
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 1024 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 6 # actually 8 was better
    embedding_aggregation = 'cls' # TODO
    n_mlp_clf_layers = 1
    n_mlp_dom_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 10
    # loss = nn.BCELoss()
    loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.BCELoss()
    validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    domain_loss = nn.BCELoss()
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    alpha = 0.1
    gradient_reversal_every_n_epoch = 1
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_DANN'

model = BERT_DANN(BERT_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

In [ ]:
# BERT - with low freq token - SoftmaxBCELoss+OptBalAccu

MAX_SENTENCE_LENGTH = 256
MIN_FREQUENCY = 40 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = True
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class BERT_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    d_ff = 1024 # = 4* d_model
    n_heads = 8
    dropout = 0.1
    e_layers = 6
    embedding_aggregation = 'cls' # TODO
    n_mlp_clf_layers = 1
    n_mlp_dom_layers = 1
    res_learning = False
    activation = nn.ReLU()
    mask_flag = False # causal mask
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 10
    # loss = nn.BCELoss()
    loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    # validation_loss = nn.BCELoss()
    validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    domain_loss = nn.BCELoss()
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    alpha = 0.1
    gradient_reversal_every_n_epoch = 1
    regularisation_loss = None
    scheduler = False
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory + f'./results/'
    name = f'BERT_DANN'

model = BERT_DANN(BERT_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

---
---
# LSTM Graveyard

In [ ]:
# LSTM
MAX_SENTENCE_LENGTH = 64
MIN_FREQUENCY = 0 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = False
PAD_FRONT = True
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class LSTM_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_recurrent_layers = 2
    bidirectional = False
    n_heads = 8
    dropout = 0.1
    n_mlp_layers = 1
    flatten = False
    activation = nn.ReLU()
    res_learning = True
    mask_flag = False # TODO
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 8
    epochs = 32
    lr = 1e-5
    patience = 4
    loss = nn.BCELoss()
    # loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    validation_loss = nn.BCELoss()
    # validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    regularisation_loss = None
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory +  f'./results/'
    name = f'LSTM_Classifier'



model = LSTM(LSTM_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)

In [ ]:
# LSTM_DANN

MAX_SENTENCE_LENGTH = 128
MIN_FREQUENCY = 0 # because 40 is statistical sample requirement
MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE = False
LOW_FREQ_TOKEN = False
CLS = True
PAD_FRONT = False
W2V_CONTEXT_WINDOW = 5 # 2 to left, 2 to right
cropped_train_data = crop_sentence_length(train_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = MAKE_CROPPED_REMAINS_INTO_NEW_INSTANCE)
cropped_val_data = crop_sentence_length(val_data, max_sentence_length =  MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_test_data = crop_sentence_length(test_data, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
cropped_future_data = crop_sentence_length(data_test, max_sentence_length = MAX_SENTENCE_LENGTH, make_cropped_remains_into_new_instance = False)
raw_token_pytorch_map = get_raw_token_pytorch_map(data = cropped_train_data, min_frequency = MIN_FREQUENCY)
train_x, train_y, val_x, val_y, test_x, test_y, train_dom, val_dom, test_dom, future_x = Data_Factory(cropped_train_data, \
                                                              cropped_val_data, \
                                                                cropped_test_data, \
                                                                    cropped_future_data, \
                                                                        MAX_SENTENCE_LENGTH, \
                                                                            raw_token_pytorch_map, \
                                                                                CLS=CLS, \
                                                                                    low_freq_special_token=LOW_FREQ_TOKEN, \
                                                                                        pad_front=PAD_FRONT)
pos_prior, neg_prior = get_distribution(train_y)
print('class prior:', pos_prior, neg_prior)
pos_dom_prior, neg_dom_prior = get_distribution(train_dom)
print('domain prior:', pos_dom_prior, neg_dom_prior)
dom1_pos_prior, dom1_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label1])
print('dom1 class prior:', dom1_pos_prior, dom1_neg_prior)
dom2_pos_prior, dom2_neg_prior = get_distribution([[0, 1] if label == 1 else [1, 0] for label in label2])
print('dom2 class prior:', dom2_pos_prior, dom2_neg_prior)
pretrain_x, pretrain_y, pretrain_mask, pretrain_dom, preval_x, preval_y, preval_mask, preval_dom = BERT_pretrain_DataFactory(train_data, val_data, SEED, raw_token_pytorch_map, MAX_SENTENCE_LENGTH)

print('---')

class LSTM_DANN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 512
    n_recurrent_layers = 2
    bidirectional = False
    n_heads = 8
    dropout = 0.1
    n_mlp_clf_layers = 1
    n_mlp_dom_layers = 1
    flatten = False
    activation = nn.ReLU()
    res_learning = True
    mask_flag = False # TODO
    train_embedding = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    batch_size = 128
    epochs = 32
    lr = 1e-5
    patience = 5
    loss = nn.BCELoss()
    # loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    validation_loss = nn.BCELoss()
    # validation_loss = nn.BCELoss(weight=torch.FloatTensor([pos_prior, neg_prior]))
    domain_loss = nn.BCELoss()
    pretrain_loss = nn.CrossEntropyLoss()
    pretrain_validation_loss = nn.CrossEntropyLoss()
    alpha = 0.1
    regularisation_loss = None
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    d_output = 2
    seq_len = MAX_SENTENCE_LENGTH
    n_unique_tokens = len(raw_token_pytorch_map)
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory + './'
    saving_address = home_directory +  f'./results/'
    name = f'LSTM_DANN'

model = LSTM_DANN(LSTM_DANN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_x, train_y, train_dom, val_x, val_y, val_dom)
print()

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_x, val_y, val_dom, best_epoch, evaluation_mode = True)
model.eval(test_x, test_y, test_dom, best_epoch, evaluation_mode = True)